In [ ]:
# Read in data
input_file = open('input_files/day5.txt', 'r')
input_data = input_file.read()[:-2]

test_data = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

In [ ]:
def parse_data(data):
    data = data.split('\n\n')
    data = [string.split('\n') for string in data]
    seeds = data[0][0].split(": ")[1]
    maps = [x[1:] for x in data[1:]]
    
    return seeds, maps


def create_rules(map_list):
    rules = []
    for mapping in map_list:
        mapping = [int(x) for x in mapping.split(" ")]
        start = mapping[1]
        stop = start + mapping[2]
        rule = mapping[0] - start
        rules.append([start, stop, rule])
    return rules


def map_source(source, rules):
    for rule in rules:
        if rule[0] <= source <= rule[1]:
            return source + rule[2]
    return source


def map_seeds_to_locations(seeds, maps):
    sources = [int(x) for x in seeds.split(" ")]
    for map_list in maps:
        rules = create_rules(map_list)
        for i, source in enumerate(sources):
            destination = map_source(source, rules)
            sources[i] = destination
    return sources


def part1(data):
    seeds, maps = parse_data(data)
    sources = map_seeds_to_locations(seeds, maps)
    return min(sources)

In [ ]:
part1(input_data)

# Part 2

In [ ]:
def parse_seeds(seeds):
    seed_list = seeds.split(" ")
    seed_list = [[int(seed_list[i]), int(seed_list[i]) + int(seed_list[i+1])] for i in range(0, len(seed_list), 2)]
    return seed_list


def create_full_rules(rules):
    cur_pos = 0
    rule_starts = [rule[0] for rule in rules]
    rule_ends = [rule[1] for rule in rules]
    while cur_pos != max(rule_ends):
        if cur_pos not in rule_starts:
            next_rule_start = min(n for n in rule_starts if n - cur_pos > 0)
            rules.append([cur_pos, next_rule_start, 0])
            cur_pos = rule_ends[rule_starts.index(next_rule_start)]
        else:
            cur_pos = rule_ends[rule_starts.index(cur_pos)]
    
    rules.append([cur_pos, None, 0])
    return rules


def find_rule(source_start, rules):
    rule_starts = [rule[0] for rule in rules]
    rule_start = max(n for n in rule_starts if source_start - n >= 0)
    rule_index = rule_starts.index(rule_start)
    
    return rules[rule_index]
    

def update_sources(sources, rules):
    destinations = []
    for source in sources:
        source_start = source[0]
        cur_pos = source_start
        source_end = source[1]
        while cur_pos != source_end:
            rule = find_rule(cur_pos, rules)
            rule_start = rule[0]
            rule_end = rule[1]
            rule_val = rule[2]
            if rule_end is None:
                destinations.append([cur_pos + rule_val, source_end + rule_val])
                cur_pos = source_end
            elif rule_end > source_end:
                destinations.append([cur_pos + rule_val, source_end + rule_val])
                cur_pos = source_end
            else:
                destinations.append([cur_pos + rule_val, rule_end + rule_val])
                cur_pos = rule_end
    return destinations


def part2(data):
    seeds, maps = parse_data(data)
    sources = parse_seeds(seeds)
    for map_list in maps:
        rules = create_rules(map_list)
        full_rules = create_full_rules(rules)
        sources = update_sources(sources, full_rules)
        
    return min([source_range[0] for source_range in sources])

In [ ]:
part2(input_data)